In [28]:
from pydantic import BaseModel
import pandas as pd
import os
import json
import utils
import time
from openai import OpenAI

In [29]:
with open("./apikey_openai.txt", "r") as file:
    apikey_openai = file.read().strip()
    os.environ["OPENAI_API_KEY"] = apikey_openai

with open("./instruction_toLLM/extract_scandal_company_name.txt", "r") as file:
    instruction = file.read().strip()

In [30]:
class scandalous_coumpany(BaseModel):
    no: int
    s_c: str
    n: str
    nn: str

class Inspectation(BaseModel):
    final_result: list[scandalous_coumpany]

In [80]:
scandal_df = pd.read_csv("./data/scandals_20200101to20241113.csv")
done_df = pd.read_csv("./data/done.csv")

scandal_df

,title,date,url
0,損保ジャパン「不祥事忘れない」 社員向けに資料展示室,2024-11-07 15:05:18,https://www.nikkei.com/article/DGXZQOUB070900X...
1,一大イベントの直前に不祥事発覚 カナデビア社長「極めて残念」,2024-11-08 15:02:29,https://www.asahi.com/articles/ASSC81SZQSC8ULF...
2,不祥事相次ぐ鹿児島県警 警察庁の見解は? 書面で回答された本部長の転出経緯と、「隠蔽の疑惑な...,2024-11-09 07:00:00,https://373news.com/_news/storyid/204175/
3,また土木部...25歳福島県職員が逮捕 盗撮か 入札巡る県職員3人の起訴に続き 再発防止の徹...,2024-11-06 19:20:00,https://www.fukushima-tv.co.jp/localnews/2024/...
4,ＩＣＣ主任検察官の不祥事調査,2024-11-12 06:36:26,https://webun.jp/articles/-/706439
...,...,...,...
15256,MOMOLAND デイジー、インタビューで番組の不正を暴露…所属事務所＆CN ENMは反論「...,2020-01-07 17:00:00,https://news.kstyle.com/article.ksn?articleNo\...
15257,【国際免許証、偽造も】外国人レンタカー利用者100万人突破！ 「不正な」国際免許証に要注意,2020-01-03 17:00:00,https://www.autocar.jp/post/453984
15258,元会長カルロス・ゴーン氏のレバノン逃亡について日産が声明を発表。その内容とは,2020-01-07 17:00:00,https://driver-web.jp/articles/detail/26573
15259,KinKi Kids・堂本光一、電子チケット転売に不快感！ 「入場後に席替えてもらうバカ」と...,2020-01-08 17:00:00,https://cyzowoman.jp/2020/01/post_264998_1.html


In [ ]:
# 不必要なデータを除く
scandal_df = scandal_df.drop_duplicates()    # df内で重複しているデータ
scandal_df = scandal_df[~scandal_df["title"].isin(done_df["title"])]    # done.csvにあるデータ
scandal_df = scandal_df.reset_index(drop=True)
scandal_df

,title,date,url
0,日野と三菱ふそうが経営統合へ向かう2つの要因 親会社のトヨタとダイムラーがCASE技術で協業,2023-06-03 05:51:35,https://toyokeizai.net/articles/-/676950
1,「メガネスーパー」、生活保護受給者向け販売を全店中止,2023-06-02 16:00:00,https://www.nikkei.com/article/DGXZQOUC02B210S...
2,かんぽ不正巡る解雇、札幌地裁「相当」 同種訴訟で判断分かれる [北海道],2023-06-05 16:00:00,https://www.asahi.com/articles/ASR65635VR65IIP...
3,＜２０２３年６月１日付組織改正＞認証申請での不正行為を受け、開発と法規認証の体制を見直し,2023-05-31 16:00:00,https://prtimes.jp/main/html/rd/p/000000307.00...
4,北海道運輸局、千歳相互観光バスを調査 部品、不正交換の疑い [北海道],2023-05-31 16:00:00,https://www.asahi.com/articles/ASR5056NJR50IIP...
...,...,...,...
6485,MOMOLAND デイジー、インタビューで番組の不正を暴露…所属事務所＆CN ENMは反論「...,2020-01-07 17:00:00,https://news.kstyle.com/article.ksn?articleNo\...
6486,【国際免許証、偽造も】外国人レンタカー利用者100万人突破！ 「不正な」国際免許証に要注意,2020-01-03 17:00:00,https://www.autocar.jp/post/453984
6487,元会長カルロス・ゴーン氏のレバノン逃亡について日産が声明を発表。その内容とは,2020-01-07 17:00:00,https://driver-web.jp/articles/detail/26573
6488,KinKi Kids・堂本光一、電子チケット転売に不快感！ 「入場後に席替えてもらうバカ」と...,2020-01-08 17:00:00,https://cyzowoman.jp/2020/01/post_264998_1.html


In [33]:
start_time = time.time()

article_ls = scandal_df["title"].to_list()
article_numbered_ls = [f"{i}.{item}" for i, item in enumerate(article_ls)]

batch_size = 100


try:
    for i in range(0, len(article_ls), batch_size):
        batch_ls = article_numbered_ls[i:i+batch_size]

        result_raw = utils.call_openai_structured_output(
                instruction= instruction,
                prompt= str(batch_ls),
                model = "gpt-4o",
                response_format= Inspectation
        )

        result_inspection = result_raw.final_result

        for item in result_inspection:
            row_index = item.no
            scandal_df.loc[row_index, 'scandal_of_company'] = item.s_c
            scandal_df.loc[row_index, 'company_name'] = item.n
            scandal_df.loc[row_index, 'related_company_name'] = item.nn

        rap = time.time()
        elapsed_time = rap - start_time
        print(f"{i} 回目のバッチ終了！：{elapsed_time:.2f} s")

        start_time = time.time()


except Exception as e:
    scandal_df.to_csv("scandal_df_partial_output.csv", index=False)
        # 処理時間を測定
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Error was captured!!!")
    print(e)


scandal_df.to_csv("./data/scandals_20200101to20241113_symbol_added.csv", index=False)

# 処理時間を測定
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Processing time: {elapsed_time:.2f} seconds")





0 回目のバッチ終了！：30.16 s
100 回目のバッチ終了！：27.03 s
200 回目のバッチ終了！：31.02 s
300 回目のバッチ終了！：41.21 s
400 回目のバッチ終了！：34.98 s
500 回目のバッチ終了！：34.09 s
600 回目のバッチ終了！：32.75 s
700 回目のバッチ終了！：30.73 s
800 回目のバッチ終了！：30.53 s
900 回目のバッチ終了！：33.27 s
1000 回目のバッチ終了！：34.83 s
1100 回目のバッチ終了！：37.66 s
1200 回目のバッチ終了！：10.40 s
1300 回目のバッチ終了！：35.32 s
1400 回目のバッチ終了！：28.97 s
1500 回目のバッチ終了！：27.92 s
1600 回目のバッチ終了！：39.95 s
1700 回目のバッチ終了！：31.54 s
1800 回目のバッチ終了！：32.36 s
1900 回目のバッチ終了！：37.87 s
2000 回目のバッチ終了！：41.52 s
2100 回目のバッチ終了！：34.88 s
2200 回目のバッチ終了！：28.38 s
2300 回目のバッチ終了！：37.18 s
2400 回目のバッチ終了！：42.08 s
2500 回目のバッチ終了！：27.89 s
2600 回目のバッチ終了！：29.87 s
2700 回目のバッチ終了！：32.83 s
2800 回目のバッチ終了！：29.88 s
2900 回目のバッチ終了！：27.95 s
3000 回目のバッチ終了！：29.98 s
3100 回目のバッチ終了！：28.34 s
3200 回目のバッチ終了！：32.25 s
3300 回目のバッチ終了！：32.27 s
3400 回目のバッチ終了！：32.25 s
3500 回目のバッチ終了！：28.41 s
3600 回目のバッチ終了！：37.34 s
3700 回目のバッチ終了！：29.39 s
3800 回目のバッチ終了！：34.03 s
3900 回目のバッチ終了！：32.06 s
4000 回目のバッチ終了！：30.49 s
4100 回目のバッチ終了！：32.54 s
4200 回目のバッチ終了！：30.41 s
4300 回目のバッチ終了！：39.01 s


## なぜか、推測されなかった箇所があるのでもう一回回す。

In [ ]:
rest_scandal_df = scandal_df[scandal_df["scandal_of_company"].isna()]
done_scandal_df = scandal_df[~scandal_df["scandal_of_company"].isna()]
rest_scandal_df

,title,date,url,scandal_of_company,company_name,related_company_name
1225,不正の懲戒、官に厳しく民に慎重 割れた最高裁判断 法務インサイド,2022-07-25 16:00:00,https://www.nikkei.com/article/DGXZQOUC064HF0W...,NaN,NaN,NaN
1226,不正アカウント480以上か 決済アプリ悪用した詐欺容疑の中国人ら,2022-07-20 16:00:00,https://www.asahi.com/articles/ASQ7N5QF2Q7NUTI...,NaN,NaN,NaN
1227,「公平性ないがしろ」 次男不正合格で文科省元局長ら有罪、賄賂認定,2022-07-20 16:00:00,https://mainichi.jp/articles/20220720/k00/00m/...,NaN,NaN,NaN
1228,＜ゴルフ＞韓国女子ゴルフ最高有望株ユン・イナ、不正行為者に転落,2022-07-26 16:00:00,https://japanese.joins.com/JArticle/293617?sec...,NaN,NaN,NaN
1229,｢個人データ不正取得｣DiDiに1600億円罰金。中国政府とIT業界に関係正常化の兆しか,2022-07-26 16:00:00,https://www.businessinsider.jp/post-257125,NaN,NaN,NaN
...,...,...,...,...,...,...
6395,報道の世界で起きた驚愕の不祥事とは──『スキャンダル』試写会60組120名ご招待,2020-01-21 17:00:00,https://www.newsweekjapan.jp/stories/culture/2...,NaN,NaN,NaN
6396,経営破綻から10年のJAL、再び拡大路線へ 飲酒続発、「気の緩み」指摘も,2020-01-16 17:00:00,https://mainichi.jp/articles/20200116/k00/00m/...,NaN,NaN,NaN
6397,同僚の更衣室盗撮、平塚市消防士長が停職 「ストレスで」,2020-01-16 17:00:00,https://www.kanaloco.jp/news/social/entry-2448...,NaN,NaN,NaN
6398,NHK大河ドラマ、“降板”スキャンダル続出！ 「薬物・強姦致傷で逮捕」「不倫スキャンダル」,2020-01-20 17:00:00,https://cyzowoman.jp/2020/01/post_266154_1.html,NaN,NaN,NaN


In [53]:
article_ls = rest_scandal_df["title"].to_list()
article_ls

['不正の懲戒、官に厳しく民に慎重 割れた最高裁判断 法務インサイド',
 '不正アカウント480以上か 決済アプリ悪用した詐欺容疑の中国人ら',
 '「公平性ないがしろ」 次男不正合格で文科省元局長ら有罪、賄賂認定',
 '＜ゴルフ＞韓国女子ゴルフ最高有望株ユン・イナ、不正行為者に転落',
 '｢個人データ不正取得｣DiDiに1600億円罰金。中国政府とIT業界に関係正常化の兆しか',
 '「Kyash」、悪用報道を受け「不正アクセスや情報漏えいによるものではない」',
 '華原朋美、不正出血が続き産婦人科や内科を受診「原因は判明した」',
 '滋賀・高島市議会、政活費不正使用の市議を告訴へ',
 '散り椿の上映スケジュール・映画情報',
 '『競争の番人』坂口健太郎が持つ“驚異的な記憶力” 山本耕史の不正がついに暴かれる',
 '華原朋美、不正出血続き、病院で検査 原因は…「自分の身体がどうなっているのか知れて本当に良かった」',
 '華原朋美、不正出血続き病院へ 血液検査で医師から注意受け食生活の改善約束「反省です」（1/2 ページ） - ねとらぼ',
 '華原朋美、不正出血が続き、子宮頸がんの検査を報告「怖いですが、現実を受け止める」',
 '華原朋美、不正出血が続き病院で検査',
 '「フルハウス」ロリ・ロックリン、大学不正入学スキャンダル後初めてテレビ出演！ 彼女が涙ながらに語ったこととは・・・？[動画あり]',
 '【独自】奈良西署、銃撃前日から不祥事発表を準備…急きょ演説決まり「対応に追われた」',
 'ロシア・不祥事・業績不振…トップ責任問う反対票増加 検証 22年総会',
 '東電が津波を軽視した背景とは？ 不祥事、経営危機…コスト重視で安全ないがしろに',
 '【スクープ】東レ・水処理事業不正会計疑惑の深層、日覺社長の保身目的で「虚偽告訴」か',
 '「野村教室」の劣等生が甲子園で叶えたい夢 1本の電話で胸の鼓動は止まらず…',
 '第12回 コーポレートガバナンスが問われた事例と企業の施策（第3篇）',
 '[写真]無理やりキス、胸やスカートの中を触られ…保険外交員の告発で愛媛県警の現役警官が書類送検',
 '「恥ずかしいからマジでやめてくれ」 女子アナとの不倫報道、バスケ選手全体の「プロ意識の低さ」露呈',
 'ネッツトヨタ千葉従業員が運転

In [58]:
article_with_number_ls = [f"{index}.{title}" for index, title in zip(rest_scandal_df.index, rest_scandal_df['title'])]
article_with_number_ls

['1225.不正の懲戒、官に厳しく民に慎重 割れた最高裁判断 法務インサイド',
 '1226.不正アカウント480以上か 決済アプリ悪用した詐欺容疑の中国人ら',
 '1227.「公平性ないがしろ」 次男不正合格で文科省元局長ら有罪、賄賂認定',
 '1228.＜ゴルフ＞韓国女子ゴルフ最高有望株ユン・イナ、不正行為者に転落',
 '1229.｢個人データ不正取得｣DiDiに1600億円罰金。中国政府とIT業界に関係正常化の兆しか',
 '1230.「Kyash」、悪用報道を受け「不正アクセスや情報漏えいによるものではない」',
 '1231.華原朋美、不正出血が続き産婦人科や内科を受診「原因は判明した」',
 '1232.滋賀・高島市議会、政活費不正使用の市議を告訴へ',
 '1233.散り椿の上映スケジュール・映画情報',
 '1234.『競争の番人』坂口健太郎が持つ“驚異的な記憶力” 山本耕史の不正がついに暴かれる',
 '1235.華原朋美、不正出血続き、病院で検査 原因は…「自分の身体がどうなっているのか知れて本当に良かった」',
 '1236.華原朋美、不正出血続き病院へ 血液検査で医師から注意受け食生活の改善約束「反省です」（1/2 ページ） - ねとらぼ',
 '1237.華原朋美、不正出血が続き、子宮頸がんの検査を報告「怖いですが、現実を受け止める」',
 '1238.華原朋美、不正出血が続き病院で検査',
 '1239.「フルハウス」ロリ・ロックリン、大学不正入学スキャンダル後初めてテレビ出演！ 彼女が涙ながらに語ったこととは・・・？[動画あり]',
 '1240.【独自】奈良西署、銃撃前日から不祥事発表を準備…急きょ演説決まり「対応に追われた」',
 '1241.ロシア・不祥事・業績不振…トップ責任問う反対票増加 検証 22年総会',
 '1242.東電が津波を軽視した背景とは？ 不祥事、経営危機…コスト重視で安全ないがしろに',
 '1243.【スクープ】東レ・水処理事業不正会計疑惑の深層、日覺社長の保身目的で「虚偽告訴」か',
 '1244.「野村教室」の劣等生が甲子園で叶えたい夢 1本の電話で胸の鼓動は止まらず…',
 '1245.第12回 コーポレートガバナンスが問われた事例と企業の施策（第3篇）',
 '1246.[写真]無理やりキ

In [64]:
for article in article_with_number_ls:

    result_raw = utils.call_openai_structured_output(
                instruction= instruction,
                prompt= article,
                model = "gpt-4o",
                response_format= Inspectation
        )

    result_inspection = result_raw.final_result

    for item in result_inspection:
        row_index = item.no
        scandal_df.loc[row_index, 'scandal_of_company'] = item.s_c
        scandal_df.loc[row_index, 'company_name'] = item.n
        scandal_df.loc[row_index, 'related_company_name'] = item.nn

    rap = time.time()
    elapsed_time = rap - start_time
    print(f"index {row_index} 終了！：{elapsed_time:.2f} s")

    start_time = time.time()





index 1225 終了！：5699.93 s
index 1226 終了！：0.94 s
index 1227 終了！：1.46 s
index 1228 終了！：1.39 s
index 1229 終了！：1.40 s
index 1230 終了！：1.23 s
index 1231 終了！：1.19 s
index 1232 終了！：1.13 s
index 1233 終了！：0.97 s
index 1234 終了！：1.04 s
index 1235 終了！：0.84 s
index 1236 終了！：0.83 s
index 1237 終了！：0.89 s
index 1238 終了！：0.91 s
index 1239 終了！：1.05 s
index 1240 終了！：1.05 s
index 1241 終了！：0.82 s
index 1242 終了！：0.97 s
index 1243 終了！：0.91 s
index 1244 終了！：0.88 s
index 1245 終了！：0.77 s
index 1246 終了！：0.90 s
index 1247 終了！：0.95 s
index 1248 終了！：0.93 s
index 1249 終了！：1.40 s
index 1250 終了！：0.98 s
index 1251 終了！：0.85 s
index 1252 終了！：0.98 s
index 1253 終了！：0.90 s
index 1254 終了！：0.78 s
index 1255 終了！：1.05 s
index 1256 終了！：0.86 s
index 1257 終了！：1.01 s
index 1258 終了！：0.88 s
index 1259 終了！：1.31 s
index 1260 終了！：1.22 s
index 1261 終了！：0.82 s
index 1262 終了！：0.82 s
index 1263 終了！：0.90 s
index 1264 終了！：0.93 s
index 1265 終了！：0.97 s
index 1266 終了！：1.07 s
index 1267 終了！：2.37 s
index 1268 終了！：1.04 s
index 1269 終了！：1.56 s
index 1

In [ ]:
scandal_df.to_csv("./data/scandals_20200101to20241113_symbol_added.csv", index=False)

[scandalous_coumpany(no=1225, s_c='N', n='最高裁', nn='-')]

In [75]:
scandal_df[scandal_df["scandal_of_company"].isna()]

,title,date,url,scandal_of_company,company_name,related_company_name


In [77]:
df_combined = pd.concat([scandal_df, done_df], ignore_index=True)
df_combined.shape

(15712, 6)

In [78]:
a = df_combined.drop_duplicates
a

<bound method DataFrame.drop_duplicates of                                                    title                 date  \
0         日野と三菱ふそうが経営統合へ向かう2つの要因 親会社のトヨタとダイムラーがCASE技術で協業  2023-06-03 05:51:35   
1                             「メガネスーパー」、生活保護受給者向け販売を全店中止  2023-06-02 16:00:00   
2                   かんぽ不正巡る解雇、札幌地裁「相当」 同種訴訟で判断分かれる [北海道]  2023-06-05 16:00:00   
3           ＜２０２３年６月１日付組織改正＞認証申請での不正行為を受け、開発と法規認証の体制を見直し  2023-05-31 16:00:00   
4                    北海道運輸局、千歳相互観光バスを調査 部品、不正交換の疑い [北海道]  2023-05-31 16:00:00   
...                                                  ...                  ...   
15707  Webテストの替え玉受験、不正受講の防止へ！『オンラインCBTソリューション』に加えて、『顔...  2022-12-16 17:00:00   
15708                     Instagram、不正アクセスを受けた場合のサポートを追加  2022-12-19 17:00:00   
15709          例外規定が不正ログインの一因に 熊本県立大が被害 2要素認証なしの名誉教授が狙われ  2022-12-14 17:00:00   
15710                        収入印紙を不正持ち出し 横浜地方法務局職員を停職９カ月  2022-12-16 17:00:00   
15711                      マイナンバー保険証新たな懸念/なりすまし・不正取得の危険性  2022-12-